# Сбор данных с Web-scraping и API для социально-научных исследований
---
## Семинары 3-4. `BeautifulSoup` и `requests`.

---
*ФСН, ОП "Политология", 2023-2024 гг.*

Лика Капустина,

lkapustina@hse.ru

**План занятия:**
1. **`requests`**;
2. **Продолжаем работу с `BeautifulSoup`**;
---

### 0. Загружаем необходимые библиотеки

In [1]:
import requests               # для отправки http запросов веб-серверам
from bs4 import BeautifulSoup # для обработки html
import pandas as pd           # для работы с таблицами
import numpy as np            # для базовых математических операций

### 1. Requests

Официальная документация библиотеки `requests` доступна по [ссылке](https://requests.readthedocs.io/en/latest/).

Для сбора данных мы с вами будем пользоваться одной основной функцией `requests` – `requests.get()`.

Её синтаксис предельно прост:

```requests.get(url)```

Выполнение этой команды вернет нам объект типа `response`. 

Если мы просто вызовем `response`, то получим тип код ответа от сервера. `Response [200]` означает, что ответ от сервера получен успешно. В других случаях мы бы получили другой код, например:
* `400` – ошибка неверного запроса;
* `403` – ошибка "запрещено";
* `404` – ошибка "не найдено";
* `500` – ошибка "внутренняя ошибка сервера";
* `502` – ошибка "неверного шлюза";
* `503` – ошибка "сервер недоступен".

In [88]:
link = 'https://www.hse.ru/ba/political/tutors'
html = requests.get(link)
html # код ответа

<Response [200]>

Как получить ошибку 404? Обратиться к несуществующей странице

In [86]:
link = 'https://www.hse.ru/ba/political/tutors/ыыы'
html = requests.get(link)
html # код ответа

<Response [404]>

Какие еще атрибуты есть у полученного объекта типа `response`? Давайте исследуем, обратившись к его атрибутам:

In [64]:
html.__attrs__ # получаем список атрибутов

['_content',
 'status_code',
 'headers',
 'url',
 'history',
 'encoding',
 'reason',
 'cookies',
 'elapsed',
 'request']

Давайте посмотрим, что у нас есть:

In [69]:
html.status_code # собственно, статус ответа сервера

200

In [71]:
html.encoding # кодировка данных

'utf-8'

In [92]:
html.elapsed # сколько времени прошло с момента отправки запроса

datetime.timedelta(seconds=1, microseconds=606133)

In [93]:
html.url # ссылка на страницу, к которой мы обращались

'https://www.hse.ru/ba/political/tutors'

И, основной для нас, атрибут `.text`. Именно его мы и будем использовать при дальнейшей работе с `BeautifulSoup`.

In [94]:
html.text # вызывается так

'<!DOCTYPE html>\n<!-- (c) Art. Lebedev Studio | http://www.artlebedev.com/ -->\n<html xmlns:perl="urn:perl" lang="ru"><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="theme-color" content="#1658DA"><meta name="viewport" content="width=device-width, initial-scale=1"><link rel="apple-touch-icon" sizes="180x180" href="/f/src/global/i/favicon/favicon_ios_180x180.png"><link rel="icon" type="image/png" sizes="32x32" href="/f/src/global/i/favicon/favicon_32x32.png"><link rel="icon" type="image/png" sizes="16x16" href="/f/src/global/i/favicon/favicon_16x16.png"><link rel="mask-icon" href="/f/src/global/i/favicon/favicon.svg" color="#0F2D69"><link rel="manifest" href="/f/src/manifest/manifest_ru.json"><meta name="msapplication-config" content="/f/src/global/i/favicon/browserconfig.xml"><link rel="shortcut icon" type="image/x-icon" href="/favicon.ico"><!--[if lt IE 10]><script src="/f/src/global/bower_compo

## 2. Вспоминаем работу с `BeautifulSoup`

На прошлом занятии мы с вами обсуждали работу с `BeautifulSoup`. Еще раз – это библиотека, которую используют для работы с языком разметки **html**. 

**Начнем с обсуждения проекта, который мы задумали**. Итак, мы хотим собрать данные о преподавателях ОП "Политология", которые ведут занятия в 2023-2024 учебном году. Итог нашего проекта – датасет, где одно наблюдение - один преподаватель, а в колонках указаны какие-то его данные (образование, стаж, и др.).

**Давайте подумаем, как нам получить желаемые данные?** Если мы хотим собрать данные по преподавателям политфака, сперва нужно собрать ссылки на их страницы. Значит, процесс сбора данных будет выглядеть так:

1. Сбор ссылок на страницы преподавателей (лежат на [странице с преподавателями](https://www.hse.ru/ba/political/tutors));
2. Сбор информации со страниц преподавателей (индивидуальные страницы преподавателей, организованные по шаблону).

### Этап 1. Сбор ссылок на страницы преподавателей ОП "Политология":

Давайте сделаем запрос к общей странице с информацией о преподавателях ОП и сохраним результат в переменную `html`. В будущем, вы будете бесконечно использовать две эти строки:

In [2]:
html = requests.get('https://www.hse.ru/ba/political/tutors')
html # <Response [200]> = все ок

<Response [200]>

Теперь наша задача - использовать функцию `BeautifulSoup`. Сохраним результат обработки html в переменную `soup`.

In [3]:
soup = BeautifulSoup(html.text) # используем атрибут .text
print(soup.prettify())

<!DOCTYPE html>
<!-- (c) Art. Lebedev Studio | http://www.artlebedev.com/ -->
<html lang="ru" xmlns:perl="urn:perl">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="#1658DA" name="theme-color"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="/f/src/global/i/favicon/favicon_ios_180x180.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/f/src/global/i/favicon/favicon_32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/f/src/global/i/favicon/favicon_16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link color="#0F2D69" href="/f/src/global/i/favicon/favicon.svg" rel="mask-icon"/>
  <link href="/f/src/manifest/manifest_ru.json" rel="manifest"/>
  <meta content="/f/src/global/i/favicon/browserconfig.xml" name="msapplication-config"/>
  <link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <!--[if lt

**Наша задача на этом этапе – собрать все ссылки на страницы преподавателей**. Давайте посмотрим на ссылки, которые можно найти в html-разметке.

Для этого используем метод `BeautifulSoup` который называется `.find_all(tag)`. Он возвращает все элементы разметки с определенным тегом в виде объекта, очень похожего на обычные списки в Python.

In [4]:
# получим объект со всеми ссылками на сайте преподавателей Вышки
soup.find_all('a') # все ссылки стандартно находятся под тегом a

[<a class="link" href="https://www.hse.ru/cookie.html" target="_blank" title="Пройти по ссылке">здесь</a>,
 <a class="link" href="https://www.hse.ru/data_protection_regulation" target="_blank" title="Пройти по ссылке">здесь</a>,
 <a class="link link_no-visited link_no-underline" href="https://www.hse.ru/en/ba/political/tutors/">EN</a>,
 <a class="control control_user control_white" href="//www.hse.ru/lkuser/" title="Личный кабинет сотрудника ВШЭ"><ins><svg class="control_svg" height="18" viewbox="0 0 17 18" width="17" xmlns="http://www.w3.org/2000/svg"><path d="M13.702 13.175c.827.315 1.486.817 1.978 1.506.492.689.738 1.467.738 2.333h-16.419c0-1.417.532-2.5 1.595-3.248.394-.276 1.358-.591 2.894-.945.945-.118 1.457-.374 1.536-.768.039-.157.059-.61.059-1.358 0-.118-.039-.217-.118-.295-.157-.157-.315-.433-.472-.827-.079-.315-.157-.787-.236-1.417-.157.039-.285-.02-.384-.177-.098-.157-.177-.364-.236-.62l-.089-.443c-.157-.866-.098-1.28.177-1.24-.118-.157-.217-.532-.295-1.122-.118-.866-.059-1

**Но как найти нужные ссылки?**

Обратимся к странице и нажнем на функцию "посмотреть код". Что мы видим? Ссылки на страницы всех преподавателей имеют определенные атрибуты:

* тег `<a>`,
* класс `'b link_dark2 link'`.

Как получить только ссылки на страницы преподавателей? Воспользоваться дополнительными возможностями метода `.find_all()`.

Чтобы получить все ссылки из html-разметки страницы, мы используем код `soup.find_all('a')`. Но если мы хотим получить объекты под тегом `a` под определенным классом, на месте второго аргумента нам нужно использовать словарь (`dictionary`) и уточнить название класса в формате: `class`: `name of class`.


```soup.find_all('a', {'class': 'b link_dark2 link'})```

In [5]:
soup.find_all('a', {'class': 'b link_dark2 link'}) # используем словарь

[<a class="b link_dark2 link" href="/mirror/org/persons/redir/305053833?pl=">Акаева Кавсарат Исламовна</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/223725168?pl=">Акоз Кемаль Киванч</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/61713365?pl=">Арбатли Эким</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/209638517?pl=">Баженов Григорий Александрович</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/93759602?pl=">Байссвенгер Мартин Эрвин</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/137284738?pl=">Баландин Юлиан Андреевич</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/224906215?pl=">Баранова Татьяна Александровна</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/160957135?pl=">Борисенко Максим Сергеевич</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/95928025?pl=">Бородкина Елена Владимировна</a>,
 <a class="b link_dark2 link" href="/mi

In [6]:
soup.find_all('a', {'class': 'link'})

[<a class="link" href="https://www.hse.ru/cookie.html" target="_blank" title="Пройти по ссылке">здесь</a>,
 <a class="link" href="https://www.hse.ru/data_protection_regulation" target="_blank" title="Пройти по ссылке">здесь</a>,
 <a class="link link_no-visited link_no-underline" href="https://www.hse.ru/en/ba/political/tutors/">EN</a>,
 <a class="link link_white no-visited" href="//www.hse.ru/search/search.html?simple=0">Расширенный поиск по сайту</a>,
 <a class="link link_no-underline link_dark2 fa-sidemenu__link" href="https://www.hse.ru/ba/political/about">О программе</a>,
 <a class="link link_no-underline link_dark2 fa-sidemenu__link" href="https://www.hse.ru/ba/political/partners">Партнеры</a>,
 <a class="link link_no-underline link_dark2 fa-sidemenu__link" href="https://www.hse.ru/adm/_/org/hse/ba/political/achievements">Достижения студентов</a>,
 <a class="link link_no-underline link_dark2 fa-sidemenu__link" href="https://www.hse.ru/ba/political/alumni">Выпускники</a>,
 <a class

Что мы можем сделать с этими элементами? Например, получить полную ссылку. Как вы видите, все эти элементы содержат только часть ссылки, например: `href="/mirror/org/persons/redir/10442646?pl="`. Это означает, что у этих элементов есть атрибут `href`, за которым содержится часть гиперссылки на страницу на сайте Вышки.

In [8]:
a_elements = soup.find_all('a', {'class': 'b link_dark2 link'})
a_elements

[<a class="b link_dark2 link" href="/mirror/org/persons/redir/305053833?pl=">Акаева Кавсарат Исламовна</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/223725168?pl=">Акоз Кемаль Киванч</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/61713365?pl=">Арбатли Эким</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/209638517?pl=">Баженов Григорий Александрович</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/93759602?pl=">Байссвенгер Мартин Эрвин</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/137284738?pl=">Баландин Юлиан Андреевич</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/224906215?pl=">Баранова Татьяна Александровна</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/160957135?pl=">Борисенко Максим Сергеевич</a>,
 <a class="b link_dark2 link" href="/mirror/org/persons/redir/95928025?pl=">Бородкина Елена Владимировна</a>,
 <a class="b link_dark2 link" href="/mi

In [15]:
empty_list = []

for i in a_elements:
    part_url = i.get('href')
    correct_url = 'https://www.hse.ru' + part_url
    empty_list.append(correct_url)

In [18]:
print(len(empty_list))

104


In [103]:
# пройдемся по первым пяти элементам
for element in soup.find_all('a', {'class': 'b link_dark2 link'})[:5]:
    print(element.get('href')) # получить часть гиперссылки можно с помощью метода .get(название атрибута)

/mirror/org/persons/redir/305053833?pl=
/mirror/org/persons/redir/223725168?pl=
/mirror/org/persons/redir/61713365?pl=
/mirror/org/persons/redir/209638517?pl=
/mirror/org/persons/redir/93759602?pl=


**Как получить нормальные ссылки?** Получить значение гиперссылки из атрибута `'href'` и склеить с доменным именем. Из чего вообще состоят ссылки? Давайте разберем на примере:

<p></p>
<center><code>https://www.webofscience.com/wos/author/rid/K-5427-2015</code></center>
<p></p>

* `https` - протокол (бывает еще http);
* `www` – префикс;
* `webofscience.com` – домен;
* `wos`, `author`, `rid` – путь/каталог.

В нашем примере наша общая ссылка выглядит так:
<p></p>
<center><code>https://www.hse.ru/mirror/org/persons/redir/305053833?pl=</code></center>
<p></p>

* `https` - протокол (бывает еще http);
* `www` – префикс;
* `hse.ru` – домен;
* `mirror`, `org`, `persons`, `redir`, `305053833?pl=` – путь/каталог.

<p></p> Чего не хватает чтобы склеить нормальные ссылки? Протокола, префикса и домена. Все нужные каталоги у нас есть, потому что то что мы получаем при обращении к атрибуту <code>href</code> выглядит вот так: <code>/mirror/org/persons/redir/305053833?pl=</code>.

In [104]:
# Сбор ссылок будет выглядеть как-то так:
links = [] # объявляем пустой список, чтобы сохранить туда данные

for element in soup.find_all('a', {'class': 'b link_dark2 link'}):
    part_link = element.get('href') # обращаемся к атрибуту href, сохраняем в переменную part_link
    full_link = f'https://www.hse.ru{part_link}' # склеиваем в одну строку полную ссылку, заранее готовим первую половину ссылки
    links.append(full_link) # добавляем ссылки в список links

In [107]:
print('В списке links находится', len(links), 'элемента(ов).')
print('Пример: ', links[0])

В списке links находится 104 элемента(ов).
Пример:  https://www.hse.ru/mirror/org/persons/redir/305053833?pl=


А какие еще атрибуты есть у объектов под тегами `a`? Давайте исследуем:

In [115]:
soup.find_all('a', {'class': 'b link_dark2 link'})[0] # обращаемся к первому элементу с помощью индексации

<a class="b link_dark2 link" href="/mirror/org/persons/redir/305053833?pl=">Акаева Кавсарат Исламовна</a>

Используйте метод `.attrs`, чтобы увидеть все атрибуты элемента.

In [116]:
soup.find_all('a', {'class': 'b link_dark2 link'})[0].attrs

{'class': ['b', 'link_dark2', 'link'],
 'href': '/mirror/org/persons/redir/305053833?pl='}

Вы также можете использовать атрибут `.text` чтобы получить текст, который стоит за гиперссылкой:

In [24]:
soup.find_all('a', {'class': 'b link_dark2 link'})[-1].text

'Юзбекова Карина Шамильевна'

In [117]:
soup.find_all('a', {'class': 'b link_dark2 link'})[0].text

'Акаева Кавсарат Исламовна'

Как получить значение определенного атрибута? Вызвать его с помощью метода `.get('href')` или вызвав имя атрибута в квадратных скобках:

In [118]:
soup.find_all('a', {'class': 'b link_dark2 link'})[0].get('href') # вариант 1

'/mirror/org/persons/redir/305053833?pl='

In [119]:
soup.find_all('a', {'class': 'b link_dark2 link'})[0]['href']     # вариант 2

'/mirror/org/persons/redir/305053833?pl='

**Помните ли вы как работают f-строки?**

* Вам нужно начать строку с символа `f`, далее открыть кавычки;
* Внутри f-строки вы пишите обычный текст;
* А когда хотите упомянуть значение какой-то переменной, открывается фигурные скобки и пишите название переменной внутри: `{name}`;

**Пример:**

```print(f'Ваше имя: {name}')```

Если сохранить в переменную `name` имя `Анастасия`, мы увидим строку вида:

```Ваше имя: Анастасия```

**Вы также можете печатать с помощью f-строк результаты определенных вычислений:**

```print(f'Результат деления 100 на 5: {100/5}')``` 

В результате будет напечатана строка: `Результат деления 100 на 5: 20.0`

In [26]:
name = 'Анастасия' # сохраняем определенное значение в переменную

# используем символ f, открываем кавычки, ссылаемся на переменные используя фигурные скобки:
print(f'Ваше имя: {name}, {name}.....{100/5}')

Ваше имя: Анастасия, Анастасия.....20.0


In [10]:
name = 'Анастасия'
print(f'Ваше имя: {name}')

Ваше имя: Анастасия


In [9]:
print(f'Результат деления 100 на 5: {100/5}')

Результат деления 100 на 5: 20.0


In [ ]:
# пример работы
name = input('Введите ваше имя: ') # сохраняем значение в переменную name
print(f'Добро пожаловать, {name}!')# печатаем строку с актуальным значением, сохраненным в name

In [ ]:
name = input('Введите ваше имя: ') # сохраняем значение в переменную name
f'Добро пожаловать, {name}!'       # печатаем строку с актуальным значением, сохраненным в name

**Давайте последуем совету с лекции и сохраним получившийся объект в файл:**

Для этого с помощью функции `pd.DataFrame()` создадим `pandas.DataFrame` и сохраним в датафрейм `links_df`.

In [28]:
empty_list

['https://www.hse.ru/mirror/org/persons/redir/305053833?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/223725168?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/61713365?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/209638517?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/93759602?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/137284738?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/224906215?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/160957135?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/95928025?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/46695224?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/160958411?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/5336818?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/314013632?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/25907562?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/223572050?pl=',
 'https://www.hse.ru/mirror/org/persons/redir/190884037?pl=',
 'https://www.h

In [109]:
links_df = pd.DataFrame(links) # внутрь помещаем объект, который используем для создания датафрейма
links_df # смотрим, что получилось

,0
0,https://www.hse.ru/mirror/org/persons/redir/30...
1,https://www.hse.ru/mirror/org/persons/redir/22...
2,https://www.hse.ru/mirror/org/persons/redir/61...
3,https://www.hse.ru/mirror/org/persons/redir/20...
4,https://www.hse.ru/mirror/org/persons/redir/93...
...,...
99,https://www.hse.ru/mirror/org/persons/redir/19...
100,https://www.hse.ru/mirror/org/persons/redir/20...
101,https://www.hse.ru/mirror/org/persons/redir/46...
102,https://www.hse.ru/mirror/org/persons/redir/31...


In [29]:
with open('some_links.txt', 'w') as file: 
    for i in empty_list:
        print(i, end='\n', file=file)

У `pandas` есть множество опций для сохранения файла. Когда мы работаем только со ссылками, предлагаю вам сохранять их в обычные текстовые документы.

In [110]:
links_df.to_csv('links_political_science_bachelor_tutors.txt', sep='\t', index=False)

Эти же данные можно прочесть с помощью метода pandas.read_csv(name of file) и убедиться, что все окей.

In [30]:
pd.read_csv('links_political_science_bachelor_tutors.txt') # в скобках - абсолютный путь к файлу или частичный

,0
0,https://www.hse.ru/mirror/org/persons/redir/30...
1,https://www.hse.ru/mirror/org/persons/redir/22...
2,https://www.hse.ru/mirror/org/persons/redir/61...
3,https://www.hse.ru/mirror/org/persons/redir/20...
4,https://www.hse.ru/mirror/org/persons/redir/93...
...,...
99,https://www.hse.ru/mirror/org/persons/redir/19...
100,https://www.hse.ru/mirror/org/persons/redir/20...
101,https://www.hse.ru/mirror/org/persons/redir/46...
102,https://www.hse.ru/mirror/org/persons/redir/31...


<center><b><font size=4>Задача №1</font></b></center>

Перед вами представлена разметка html, в которой сохранена информация о ссылках на страницы на сайте Вышки. Выполните следующие задания:

In [31]:
html = '''
<a class="link fa-person__name" href="//www.hse.ru/staff/dariamaltseva">
 							Мальцева Дарья Васильевна
 						</a>,
<a data-value="talentpool" href="javascript:void(0);" class="navigation__link category__filter">Академический кадровый резерв</a>
 <a class="link fa-person__name" href="//www.hse.ru/org/persons/510416125">
 							Павлова Ирина Анатольевна
 						</a>,
 <a class="link fa-person__name" href="//www.hse.ru/org/persons/563386642">
 							Заболотский Александр Викторович
 						</a>,
<a data-value="admcrv" href="javascript:void(0);" class="navigation__link category__filter">Административный кадровый резерв</a>
 <a class="link fa-person__name" href="//www.hse.ru/org/persons/63729">
 							Климов Иван Александрович
 						</a>,
 <a class="link fa-person__name" href="//www.hse.ru/org/persons/305084525">
 							Капустина Лика Владимировна
 						</a>,
<a data-value="besttutor" href="javascript:void(0);" class="navigation__link category__filter">Лучшие преподаватели</a>
 <a class="link fa-person__name" href="//www.hse.ru/org/persons/225564557">
 							Оганян Артём Ашотович
 						</a>,
<a data-value="allowance" href="javascript:void(0);" class="navigation__link category__filter">Получатели академических надбавок</a>
 <a class="link fa-person__name" href="//www.hse.ru/staff/beylina">
 							Стегний Елена Анатольевна
 						</a>
'''

1. Преобразуйте `html` в объект `BeautifulSoup`. Сохраните его в переменную `soup`.

In [7]:
# YOUR CODE HERE

2. Сохраните все ссылки на **страницы сотрудников** под тегом `"a"` в список `new_links`.

In [6]:
# YOUR CODE HERE

3. Напечатайте тексты, за которыми стоят гиперссылки:

In [5]:
# YOUR CODE HERE

4. Избавьтесь от лишних символов, которые вы видите. Используйте метод `.strip()` или `.replace()`.

In [3]:
# YOUR CODE HERE

5. Напечатайте гиперссылки (только часть элементов под тегами a) из вашего списка `new_links`.

In [2]:
# YOUR CODE HERE

6. Создайте список `correct_links`, куда вы сохраните полные ссылки. 

In [1]:
# YOUR CODE HERE

7. Проверьте, что вы создали ссылки корректно: с помощью функции `requests.get()` отправьте запрос к первой ссылке и проинтерпретируйте ответ:

In [ ]:
# YOUR CODE HERE